# Recent Screenshots


In [ ]:
import requests
import time
import urllib.parse
import datetime
from IPython.core.display import display, HTML, clear_output

while True:
    try:
        r = requests.get("http://192.168.45.91:90/intranet/api/stats/crawler/recent-activity", timeout=5 )
    except:
        # Probably a time-out:
        pass
    if r.status_code == 200:
        display(HTML('<div>'))
        html = '<div>'
        for ss in r.json()['screenshots']:
            params = {'url': ss[0], 'target_date': ss[1]}
            url = "http://192.168.45.91:90/intranet/api/screenshots/?" + urllib.parse.urlencode(params)
            html += '<a href="" style="float: left;"><img src="%s" style="max-height:150px;"/></a>' % url
        html += '</div>'
        display(HTML(html))
    else:
        display(HTML('ERROR at %s' % datetime.datetime.now()))
        
    clear_output(wait=True)
    time.sleep(10)
